In [ ]:
#default_exp

In [ ]:
#hide
from nbdev.showdoc import show_doc

# Example - Image classification w Chitra
> Training Image classification model for Cats vs Dogs Kaggle dataset.

<a href="https://colab.research.google.com/github/aniketmaurya/chitra/blob/master/examples/example_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install chitra

`pip install --upgrade chitra==0.0.20`

In [ ]:
!pip install chitra -q

## import functions and classes
### Dataset Class
Dataset class has API for loading `tf.data`, image augmentation and progressive resizing.

### Trainer
The Trainer class inherits from tf.keras.Model, it contains everything that is required for training. It exposes trainer.cyclic_fit method which trains the model using Cyclic Learning rate discovered by Leslie Smith.


In [ ]:
import tensorflow as tf
from chitra.datagenerator import Dataset
from chitra.trainer import Trainer, create_cnn

from PIL import Image

In [ ]:
BS = 16
IMG_SIZE_LST = [(128,128), (160, 160), (224,224)]
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
def tensor_to_image(tensor):
    return Image.fromarray(tensor.numpy().astype('uint8'))

In [ ]:
# !kaggle datasets download -d chetankv/dogs-cats-images
# !unzip -q dogs-cats-images.zip

In [ ]:
ds = Dataset('dog vs cat/dataset/training_set', image_size=IMG_SIZE_LST)

In [ ]:
image, label = ds[0]
print(label)
tensor_to_image(image).resize((224,224))

## Create Trainer

Train imagenet pretrained MobileNetV2 model with cyclic learning rate and SGD optimizer.

In [ ]:
trainer = Trainer(ds, create_cnn('mobilenetv2', num_classes=2))

In [ ]:
trainer.summary()

In [ ]:
trainer.compile2(batch_size=BS,
                 optimizer='sgd',
                 lr_range=(1e-4, 1e-2),
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=['binary_accuracy'])

In [ ]:
trainer.cyclic_fit(10, batch_size=BS)

Trainer also supports the regular keras `model.fit` api using `trainer.fit`

Train the same model **without cyclic learning rate**:

In [ ]:
trainer = Trainer(ds, create_cnn('mobilenetv2', num_classes=2))
trainer.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['binary_accuracy'])

In [ ]:
data = ds.get_tf_dataset().map((lambda x,y: (x/127.5-1.0, y)), AUTOTUNE).batch(BS).prefetch(AUTOTUNE)

trainer.fit(data,
            epochs=10)

# What does model focus on while making a prediction?
`chitra.trainer.InterpretModel` class creates GradCAM and GradCAM++ visualization in no additional code!

In [ ]:
from chitra.trainer import InterpretModel
import random
model_interpret = InterpretModel(True, trainer)

In [ ]:
image_tensor = random.choice(ds)[0]
image = tensor_to_image(image_tensor)
model_interpret(image, auto_resize=False)